# Import Library

In [1]:
import ortools
import sys
import time
from math import ceil
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

# Read Data

In [2]:
filename = 'C:/Users/Admin/Desktop/Capstone-Project-Optimization/Project Optimization/fix_data/data2.txt'
def readData(filename):
  with open(filename) as f:
    content = [[int(j) for j in i.split()] for i in f.read().splitlines()]
  N, M, d, c, K = content[0][0], content[0][1], content[1], content[2], content[3][0]
  p = [[content[4 + i][0], content[4 + i][1]] for i in range(K)]
  print(f'N = {N}', f'M = {M}', sep = ' ')
  print(f'd = {d}', f'c = {c}', f'K = {K}', f'p = {p}', sep = '\n')
  return N, M, d, c, K, p
N, M, d, c, K, p = readData(filename)
d.insert(0,0)
c.insert(0,0)

N = 50 M = 10
d = [54, 51, 54, 53, 52, 76, 68, 67, 71, 75, 62, 68, 69, 65, 66, 54, 51, 49, 52, 52, 69, 69, 69, 69, 64, 43, 44, 52, 45, 51, 51, 56, 55, 51, 57, 75, 76, 81, 84, 80, 55, 48, 53, 48, 47, 72, 64, 73, 71, 71]
c = [57, 76, 71, 54, 72, 52, 58, 84, 56, 73]
K = 491
p = [[1, 2], [1, 4], [1, 7], [1, 8], [1, 10], [1, 12], [1, 14], [1, 15], [1, 18], [1, 25], [1, 27], [1, 28], [1, 30], [1, 32], [1, 35], [1, 37], [1, 39], [1, 40], [1, 43], [1, 44], [1, 45], [1, 50], [2, 3], [2, 6], [2, 9], [2, 10], [2, 18], [2, 24], [2, 28], [2, 31], [2, 33], [2, 34], [2, 35], [2, 36], [2, 43], [2, 44], [2, 45], [2, 48], [3, 5], [3, 6], [3, 7], [3, 9], [3, 11], [3, 14], [3, 15], [3, 19], [3, 21], [3, 24], [3, 26], [3, 30], [3, 31], [3, 35], [3, 36], [3, 39], [3, 42], [3, 46], [3, 49], [3, 50], [4, 6], [4, 7], [4, 10], [4, 12], [4, 16], [4, 17], [4, 22], [4, 36], [4, 38], [4, 43], [4, 46], [5, 6], [5, 9], [5, 14], [5, 16], [5, 18], [5, 21], [5, 22], [5, 24], [5, 27], [5, 28], [5, 32], [5, 33], [5, 34], 

# Algorithms

## Mixed Integer Programming

In [3]:
def printSolution():
  solution = []
  for i in range(1,obj_value+1):
    for j in range(M+1):
      if solution_matrix[i][j] != -1:
        solution.append([solution_matrix[i][j], i, j])
  solution = sorted(solution, key= lambda x: x[0])
  for i in solution:
    print(*i)
    
# Instantiate a MIP mip_solver
mip_solver = pywraplp.Solver.CreateSolver('SCIP')

# Infinity
INF = mip_solver.infinity()

# Define variables

# Variable x[i][j][k]
x = [[[mip_solver.IntVar(0, 1, f'x[{i}][{j}][{k}]') for i in range(N+1)] for j in range(M+1)] for k in range(N+1)]

# Variable y
y = mip_solver.IntVar(0, N , 'y')

# Define constraints

# Constraint 1: Pairs of conflicting courses may not be put in the same period
for i in range(K):
  u, v = p[i][0], p[i][1]
  for k in range(1,N+1):
    constraint = mip_solver.Constraint(0, 1)
    for j1 in range(1,M+1):
      for j2 in range(1,M+1):
        if j1 != j2:
          constraint.SetCoefficient(x[u][j1][k], 1)
          constraint.SetCoefficient(x[v][j2][k], 1)

# Constraint 2: An course room may be assigned at most one course in a period
for j in range(1,M+1):
  for k in range(1,N+1):
    constraint = mip_solver.Constraint(0, 1)
    for i in range(1,N+1):
      constraint.SetCoefficient(x[i][j][k], 1)

# Constraint 3: The number of periods (k.x[i,j,k] - y <= 0)
for i in range(1,N+1):
  for j in range(1,M+1):
    for k in range(1,N+1):
      constraint = mip_solver.Constraint(-INF, 0)
      constraint.SetCoefficient(y, -1)
      constraint.SetCoefficient(x[i][j][k], k)

# Constraint 4: A course may be conducted at most one time in an course room
for i in range(1,N+1):
  constraint = mip_solver.Constraint(1, 1)
  for j in range(1,M+1):
    for k in range(1,N+1):
      constraint.SetCoefficient(x[i][j][k], 1)

# Constraint 5: A course n_i must be put into a room m_j with capacity c(j)
for i in range(1,N+1):
  for j in range(1,M+1):
    constraint = mip_solver.Constraint(0, c[j])
    for k in range(1,N+1):
      constraint.SetCoefficient(x[i][j][k], d[i])

# Define objective
obj = mip_solver.Objective()
obj.SetCoefficient(y, 1)
obj.SetMinimization()

mip_solver.SetTimeLimit(30000)

# Solve and count elapsed time
start_time = time.time()
status = mip_solver.Solve()
end_time = time.time()



# Print solution
if status == mip_solver.OPTIMAL or status == mip_solver.FEASIBLE:
  obj_value = int(obj.Value())
  solution_matrix = []
  for i in range(obj_value+1):
    solution_matrix.append([-1 for _ in range(M+1)])
  for k in range(1,obj_value+1):
    for j in range(1,M+1):
      for i in range(1,N+1):
        if x[i][j][k].solution_value() == 1:
            solution_matrix[k][j] = i
  printSolution()
else:
  print('No solution found.')

print(f'The minimum period needed is: {obj_value}, equivalent to {ceil(obj_value/4)} days')
print(f'Used time: {1000*(end_time - start_time)} milliseconds')

1 11 8
2 17 5
3 3 9
4 1 4
5 2 5
6 14 2
7 2 2
8 18 2
9 13 10
10 5 2
11 1 8
12 9 3
13 9 8
14 17 2
15 4 2
16 10 5
17 2 10
18 16 6
19 6 6
20 1 9
21 6 10
22 16 5
23 18 8
24 15 10
25 17 8
26 15 3
27 16 7
28 7 6
29 2 3
30 14 1
31 2 6
32 3 8
33 18 7
34 5 10
35 7 2
36 8 8
37 16 8
38 12 8
39 13 8
40 14 8
41 1 5
42 2 8
43 18 3
44 14 9
45 3 2
46 15 5
47 13 3
48 4 10
49 5 5
50 17 3
The minimum period needed is: 18, equivalent to 5 days
Used time: 29641.263484954834 milliseconds


## Constraint Programming

In [4]:
def printSolution():
  solution = []
  for i in range(1,obj_value+1):
    for j in range(M+1):
      if solution_matrix[i][j] != -1:
        solution.append([solution_matrix[i][j], cp_solver.Value(x[i]),j])
  solution = sorted(solution, key= lambda x: x[0])
  for i in solution:
    print(*i)
# Constraint Programming

# Initianate a model
model = cp_model.CpModel()

# Define variables

# Variable x[i]
x = [model.NewIntVar(1, N, f'x[{i}]') for i in range(N+1)]

# Variable y[i][j]
y = [[model.NewIntVar(0, 1, f'y[{i}][{j}]') for j in range(M+1)] for i in range(N+1)]

# Define constraints

# Constraint 1: Pairs of conflicting courses may not be put in the same period 
for pair in p:
  model.Add(x[pair[0]] != x[pair[1]])

# Constraint 2: An course room may be assigned at most one course in a period
for i in range(1,N+1):
  model.Add(sum(y[i]) == 1)

# Constraint 3: Courses with same period may not share an course room
for j in range(1,M+1):
  for i1 in range(1,N):
    for i2 in range(i1 + 1, N+1):
      b = model.NewBoolVar(f'b[{j}][{i1}][{i2}]')
      model.Add(y[i1][j] + y[i2][j] <= 1).OnlyEnforceIf(b)
      model.Add(x[i1] == x[i2]).OnlyEnforceIf(b)
      model.Add(x[i1] != x[i2]).OnlyEnforceIf(b.Not())

# Constraint 4: A course n_i must be put into a room m_j with adequate capacity c(j)
for i in range(1,N+1):
  model.Add(sum([y[i][j] * c[j] for j in range(M+1)]) >= d[i])

# Define objective
cp_obj = model.NewIntVar(1, N, 'obj')
model.AddMaxEquality(cp_obj, x)
model.Minimize(cp_obj)

# Instantiate a CP solver 
cp_solver = cp_model.CpSolver()
cp_solver.parameters.max_time_in_seconds = 40.0

# Solve and count used time
start_time = time.process_time()
status = cp_solver.Solve(model)
end_time = time.process_time()

# Print solution
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  obj_value = int(cp_solver.Value(cp_obj)) # Objective value
  solution_matrix = []
  for i in range(obj_value+1):
    solution_matrix.append([-1 for _ in range(M+1)])
  for i in range(1, N+1):
    for j in range(1, M+1):
      if cp_solver.Value(y[i][j]) == 1:
        solution_matrix[int(cp_solver.Value(x[i]))][j] = i
        break
  printSolution()
else:
  print('Not found solution.')

print(f'The minimum period needed is: {obj_value}, equivalent to {ceil(obj_value/4)} days')
print(f'Used time: {1000*(end_time - start_time)} milliseconds')

1 2 7
2 1 6
3 3 9
4 5 7
5 4 4
6 2 2
7 1 3
8 3 3
9 5 3
10 4 2
11 2 5
12 1 10
13 3 10
14 5 10
15 4 3
16 2 9
17 1 9
18 3 6
19 5 9
20 4 6
21 2 3
22 1 5
23 3 5
24 5 5
25 4 10
26 2 6
27 1 4
28 3 4
29 5 6
30 4 1
31 2 4
32 1 7
33 3 7
34 5 1
35 4 7
36 2 8
37 1 8
38 3 8
39 5 8
40 4 8
41 2 1
42 1 1
43 3 1
44 5 4
45 4 9
46 2 10
47 1 2
48 3 2
49 5 2
50 4 5
The minimum period needed is: 5, equivalent to 2 days
Used time: 7546.875 milliseconds


## Heuristic 1

In [5]:
# List of (capacity, room) are sorted by capacity in ascending order 
sorted_c = sorted([(c[i], i) for i in range(1,M+1)])
sorted_c.insert(0,0)
# Conflicts
conflicts = {} # conflicts[i] = list of courses that cannot be administered in the same period as course i+1
for pair in p:
    conflicts.setdefault(pair[0], []).append(pair[1])
    conflicts.setdefault(pair[1], []).append(pair[0])


result = [0,[-1] * (M+1)] # initiate with first period 
                      # Result[i, k] = course exam administered in period i+1 and room k+1
def heuristic_2():
  for exam in range(1,N+1): #sequentially assign a period and a room to each course
    nextCourse = False
    for period in range(1,len(result) + 1): #consider existing periods first
      if period == len(result):
        #if this exam cannot be held in any existing period, create a new period
        result.append([-1] * (M+1)) # new period with M rooms
      not_ThisPeriod = False
      if exam in conflicts:
        for otherCourse in result[period]:
          if otherCourse in conflicts[exam]:
            not_ThisPeriod = True
            break
        if not_ThisPeriod == True:
          continue 
      for room in range(1,M+1): #consider smaller rooms first to save bigger ones for other courses
        capacity = sorted_c[room][0]
        roomIndex = sorted_c[room][1]
        if result[period][roomIndex] == -1 and capacity >= d[exam]:
          result[period][roomIndex] = exam #stick this exam to this period and room
          nextCourse = True
          break
      if nextCourse == True:
        break
  return len(result)-1, result

start_time = time.time()
obj_value, solution_matrix = heuristic_2()
end_time = time.time()

# Result

res = []
for period in range(1,len(result)):
    for room in range(1,M+1):
        if result[period][room] != -1:
            res.append([result[period][room], period, room])
res = sorted(res, key= lambda x: x[0])
for i in res:
    print(*i)
    
print(f'The minimum period needed is: {obj_value}, equivalent to {ceil(obj_value/4)} days')
print(f'Used time: {1000*(end_time - start_time)} milliseconds')

1 1 4
2 2 6
3 1 9
4 2 4
5 2 9
6 3 2
7 3 3
8 2 3
9 3 5
10 4 2
11 3 10
12 5 3
13 1 3
14 4 3
15 4 5
16 6 4
17 3 6
18 5 6
19 7 6
20 1 6
21 6 3
22 5 5
23 1 5
24 7 3
25 2 5
26 6 6
27 5 4
28 7 4
29 2 1
30 2 7
31 6 9
32 3 9
33 1 1
34 8 6
35 9 1
36 6 2
37 5 2
38 8 8
39 4 8
40 9 8
41 6 1
42 10 6
43 11 4
44 4 6
45 4 4
46 6 5
47 10 3
48 1 10
49 7 5
50 9 3
The minimum period needed is: 11, equivalent to 3 days
Used time: 0.7159709930419922 milliseconds


## Heuristic 2

In [6]:
conflicts = {} #conflicts[i] = list of exams that cannot be administered in the same period as exam i+1
for pair in p:
    conflicts.setdefault(pair[0], []).append(pair[1])
    conflicts.setdefault(pair[1], []).append(pair[0])

sortedExams = sorted([(d[i], i) for i in range(N)], reverse=True) #sort exams in ascending order of capacity

schedule = [] #schedule[i, k] = exam administered in period i+1 and room k+1
period = 0
start_time = time.time()
while sortedExams: #sequentially fill each period with as many exams as possible until all exams have been scheduled
    schedule.append([None] * M)
    for room in range(M):
        for exam in sortedExams: #consider more popular exams first
            if exam[0] <= c[room]: #if a hall has adequate capacity
                #check if any exam already scheduled in this period has common candidates with the one being considered
                noConflict = True
                if exam[1] in conflicts:
                    for scheduledExam in schedule[period]:
                        if scheduledExam in conflicts[exam[1]]:
                            noConflict = False
                            break
                if noConflict: #schedule exam in period and room and remove from list of exams to schedule
                    schedule[period][room] = exam[1]
                    sortedExams.remove(exam)
                    break
    period += 1
#PRINT RESULT
end_time = time.time()

solution = []
for pe in range(period): #print schedule by period
    for room in range(M):
        exam = schedule[pe][room]
        conflictsOfThisExam = [e + 1 for e in conflicts.get(exam, [])]
        if exam != None:
            solution.append([exam+1,pe+1,room+1])
solution = sorted(solution, key= lambda x: x[0])
for i in solution:
    print(*i)

print(f'The minimum period needed is: {obj_value}, equivalent to {ceil(obj_value/4)} days')
print(f'Used time: {1000*(end_time - start_time)} milliseconds')

1 1 1
2 3 5
3 2 6
4 4 5
5 2 5
6 6 5
7 1 3
8 6 4
9 4 6
10 1 4
11 4 3
12 3 9
13 6 3
14 5 4
15 7 6
16 7 4
17 1 5
18 8 3
19 5 6
20 7 2
21 6 2
22 3 4
23 1 6
24 4 4
25 7 3
26 1 9
27 3 8
28 9 2
29 5 5
30 6 7
31 1 7
32 3 7
33 2 2
34 4 2
35 8 2
36 1 2
37 3 3
38 2 3
39 4 9
40 7 9
41 8 9
42 3 2
43 6 6
44 5 2
45 7 5
46 8 4
47 3 6
48 9 3
49 5 3
50 2 4
The minimum period needed is: 11, equivalent to 3 days
Used time: 1.5816688537597656 milliseconds


## Heuristic 3

In [7]:
conflicts = {} #conflicts[i] = list of exams that cannot be administered in the same period as exam i+1
for pair in p:
    conflicts.setdefault(pair[0], []).append(pair[1])
    conflicts.setdefault(pair[1], []).append(pair[0])

sortedRooms = sorted([(c[i], i) for i in range(1,M+1)]) #sort halls in ascending order of capacity
sortedRooms.insert(0,0)
result = [0,[None] * (M+1)] #result[i, k] = exam administered in period i+1 and room k+1
def heuristic_3():
    for exam in range(1,N+1): #sequentially assign a period and a room to each exam
        stop = False
        for period in range(1,len(result)+1): #consider existing periods first 
            for room in range(1,M+1): #consider smaller halls first to save bigger ones for other exams
                capacity = sortedRooms[room][0]
                roomIndex = sortedRooms[room][1]
                if capacity >= d[exam] and result[period][roomIndex] == None:
                    noConflict = True
                    if exam in conflicts:
                        for otherExam in result[period]:
                            if otherExam in conflicts[exam]:
                                noConflict = False
                                break
                    if noConflict:
                        result[period][roomIndex] = exam
                        print(exam, period, room, sep=' ') #print schedule by exam
                        stop = True
                        break
            if stop:
                break
            if period == len(result) - 1:
                #if this exam cannot be held in any existing period, set up a new period
                result.append([None] * (M+1))
    return len(result) - 1, result

#PRINT RESULT
start_time = time.time()
obj_value, solution_matrix = heuristic_3()
end_time = time.time()

print(f'The minimum period needed is: {obj_value}, equivalent to {ceil(obj_value/4)} days')
print(f'Used time: {1000*(end_time - start_time)} milliseconds')

1 1 2
2 2 1
3 1 3
4 2 2
5 2 3
6 3 9
7 3 6
8 2 6
9 3 7
10 4 9
11 3 8
12 5 6
13 1 6
14 4 6
15 4 7
16 6 2
17 3 1
18 5 1
19 7 1
20 1 1
21 6 6
22 5 7
23 1 7
24 7 6
25 2 7
26 6 1
27 5 2
28 7 2
29 2 4
30 2 5
31 6 3
32 3 3
33 1 4
34 8 1
35 9 4
36 6 9
37 5 9
38 8 10
39 4 10
40 9 10
41 6 4
42 10 1
43 11 2
44 4 1
45 4 2
46 6 7
47 10 6
48 1 8
49 7 7
50 9 6
The minimum period needed is: 11, equivalent to 3 days
Used time: 15.69223403930664 milliseconds
